In [14]:
# fireworks_api_key = ""

In [15]:
import glob
for filename in glob.glob("resources/*.html"):
    print(filename)
    break
    
with open(filename) as f:
    srr = f.read()

resources/Anthropic-3bfd8fb0-7841-498e-9672-40a22b2131a9.html


In [16]:
from bs4 import BeautifulSoup

In [17]:
text = BeautifulSoup(srr).get_text()

In [18]:
import openai
import json
import textwrap

client = openai.OpenAI(
    base_url = "https://api.fireworks.ai/inference/v1",
    api_key = fireworks_api_key
)

# Not recommended to change the system prompt, 
messages = [
    {"role": "system", "content": "You are a helpful assistant with access to functions."},
    {"role": "user", "content": "Please index the following job opening:\n\n" + text},
]

tools = [
    {
        "type": "function",
        "function": {
            # name of the function 
            "name": "index_job_opening",
            # a good, detailed description for what the function is supposed to do
            "description": "Index the job opening.",
            # a well defined json schema: https://json-schema.org/learn/getting-started-step-by-step#define
            "parameters": {
                # for OpenAI compatibility, we always declare a top level object for the parameters of the function
                "type": "object",
                # the properties for the object would be any arguments you want to provide to the function
                "properties": {
                    "company": {
                        "type": "string",
                        "description": "The company with the job opening."
                    },
                    "job_title": {
                        "type": "string",
                        "description": "The title of the job.",
                    },
                    "years_of_experience_required": {
                        "type": "integer", 
                        # If the model does not understand how it is supposed to fill the field, a good description goes a long way 
                        "description": "The minimum number of years of experience required for the role.",
                    },
                    "technical_skills_required": {
                        "type": "array",
                        "description": textwrap.dedent("""
                            The technical skills required (such as Python, C++).
                            NEVER include tools (Microsoft Word, VSCode).
                            NEVER include soft skills (communication, presentation).
                            This field could be empty if the position is not a technical position.
                        """.strip()),
                        "items": {
                            "type": "string"
                        },
                    },
                    "is_remote": {
                        "type": "string",
                        "description": "Whether the job can be done completely remotely.",
                    },
                    "is_intern": {
                        "type": "string",
                        "description": "Whether the job opening is an intern position.",
                    },
                    "is_new_grad": {
                        "type": "string",
                        "description": "Whether the job is specified to be open to a graduate fresh out of college.",
                    },
                    "is_manager": {
                        "type": "string",
                        "description": "Whether the job requires managing reports.",
                    },
                },
                # You can specify which of the properties from above are required
                # for more info on `required` field, please check https://json-schema.org/understanding-json-schema/reference/object#required
                "required": ["is_remote", "years_of_experience_required", "company"],
            },
        },
    }
]

chat_completion = client.chat.completions.create(
    model="accounts/fireworks/models/firefunction-v1",
    messages=messages,
    tools=tools,
    temperature=0.1
)
print(chat_completion.choices[0].message.model_dump_json(indent=4))

{
    "content": null,
    "role": "assistant",
    "function_call": null,
    "tool_calls": [
        {
            "id": "call_NXdkD5HYYHuVPWd7enXBuUFx",
            "function": {
                "arguments": "{\"company\": \"Anthropic\", \"job_title\": \"Research Engineer, Product\", \"years_of_experience_required\": 5, \"technical_skills_required\": [\"Python\", \"C++\", \"Pytorch\"], \"is_remote\": \"Hybrid\", \"is_intern\": \"No\", \"is_new_grad\": \"No\", \"is_manager\": \"Yes\"}",
                "name": "index_job_opening"
            },
            "type": "function",
            "index": 0
        }
    ]
}


In [19]:
chat_completion.choices[0].message.tool_calls[0].function.arguments

'{"company": "Anthropic", "job_title": "Research Engineer, Product", "years_of_experience_required": 5, "technical_skills_required": ["Python", "C++", "Pytorch"], "is_remote": "Hybrid", "is_intern": "No", "is_new_grad": "No", "is_manager": "Yes"}'